In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import wot

Load transport map model and cell sets

In [2]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

Convert cell sets indicating whether a cell at time t is in a cell set

In [3]:
populations = tmap_model.population_from_cell_sets(cell_sets, at_time=18)
# print cell size sizes
for name in populations:
    p = populations[name].p
    print('{}, {}/{}'.format(name, np.count_nonzero(p), len(p)))

IPS, 412/3799
Stromal, 1239/3799
Neural, 819/3799
Epithelial, 503/3799
Epithelial_clust, 1013/3799
Trophoblast, 292/3799
OPC, 141/3799
Astrocyte, 112/3799
Neuron, 122/3799
RG_Gdf10, 144/3799
RG_Id3, 160/3799
RG_Neurog2, 21/3799
RG, 255/3799
OtherNeural, 390/3799
TrophoLarger, 1305/3799


Compute trajectories. Trajectory dataset has cells on rows and cell sets (trajectories) on columns

In [4]:
trajectory_ds = tmap_model.compute_trajectories(populations)

Load embedding coordinates

In [5]:
coord_df = pd.read_csv('data/fle_coords.txt', sep='\t', index_col=0)
trajectory_ds.obs = trajectory_ds.obs.join(coord_df)

Visualize trajectories

In [8]:
trajectory_dropdown = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectories:'
)

def update_trajectory_vis(name):
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title(name)
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(trajectory_ds.obs['x'], trajectory_ds.obs['y'], c=trajectory_ds[:, name].X,
                   s=4, marker=',', edgecolors='none')
    plt.colorbar().ax.set_title('Trajectory')

widgets.interact(update_trajectory_vis, name=trajectory_dropdown)

interactive(children=(Dropdown(description='Trajectories:', options=('IPS', 'Stromal', 'Neural', 'Epithelial',…

<function __main__.update_trajectory_vis(name)>

Compute and visualize log odds

In [11]:
trajectory_dropdown1 = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectory 1:'
)
trajectory_dropdown2 = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectory 2:'
)

def update_log_odds_vis(name1, name2):
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title('Log odds, {} vs. {}'.format(name1, name2))
    p = np.log((trajectory_ds[:, name1].X + 1e-9) / (trajectory_ds[:, name2].X + 1e-9))
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(trajectory_ds.obs['x'], trajectory_ds.obs['y'], c=p,
                   s=4, marker=',', edgecolors='none')
    plt.colorbar().ax.set_title('Log Odds')

widgets.interact(update_log_odds_vis, name1=trajectory_dropdown1, name2=trajectory_dropdown2)

interactive(children=(Dropdown(description='Trajectory 1:', options=('IPS', 'Stromal', 'Neural', 'Epithelial',…

<function __main__.update_log_odds_vis(name1, name2)>